# OME-Zarr ROI processing
This notebook shows how to load a whole OME-Zarr image, apply some processing to it and store the results as a new label image into the OME-Zarr. 

First run the imports & activate the helper functions.

There are 4 prcoessing steps:  
1a) Load an intensity image  
1b) Alternatively, load an existing label image  
2a) Process the image to create a new label image  
2b) Change the label image (e.g. interactively in napari)  
3a) Save the new label image
3b) Save the changed label image to OME-Zarr  
4) Optionally save masking ROI tables

In [ ]:
import zarr
import dask.array as da
import numpy as np
from skimage.measure import label
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_holes, remove_small_objects
import napari
from pathlib import Path
import copy

In [ ]:
# Create a helper function to calculate masking ROI tables based on label_image
from fractal_tasks_core.roi import (
    array_to_bounding_box_table,
)
from fractal_tasks_core.roi import empty_bounding_box_table
import pandas as pd
import anndata as ad
import logging
from fractal_tasks_core.tables import write_table
logger = logging.getLogger(__name__)

from fractal_tasks_core.labels import prepare_label_group
from fractal_tasks_core.ngff.zarr_utils import load_NgffImageMeta 
from fractal_tasks_core.pyramids import build_pyramid

def save_masking_roi_table(
    masking_roi_df: pd.DataFrame, 
    zarr_url: str, 
    output_ROI_table: str,
    output_label_name: str,
    overwrite: bool = True,
):
    """
    Saves a masking ROI table to the OME-Zarr

    masking_roi_df: Dataframe containing the masking rois, e.g. generated by 
        `array_to_bounding_box_table`. 
    zarr_url: path to the zarr image (e.g. "/path/to/myplate.zarr/B/03/0")
    output_ROI_table: Name of the ROI table to be saved
    output_label_name: Name of the label image that has already been written 
        to the Zarr and contains the labels for the masking ROI table
    overwrite: Whether an existing roi table with name `output_ROI_table` 
        should be overwritten.
    """
    masking_roi_df.index = masking_roi_df["label"].astype(str)

    # Extract labels & drop them from dataframe
    labels = pd.DataFrame(masking_roi_df["label"].astype(str))
    masking_roi_df.drop(labels=["label"], axis=1, inplace=True)

    # Convert all to float (warning: some would be int, in principle)
    bbox_dtype = np.float32
    masking_roi_df = masking_roi_df.astype(bbox_dtype)

    # Convert to anndata
    bbox_table = ad.AnnData(masking_roi_df, dtype=bbox_dtype)
    bbox_table.obs = labels

    # Write to zarr group
    image_group = zarr.group(zarr_url)
    logger.info(
        "Now writing bounding-box ROI table to "
        f"{zarr_url}/tables/{output_ROI_table}"
    )
    table_attrs = {
        "type": "masking_roi_table",
        "region": {"path": f"../labels/{output_label_name}"},
        "instance_key": "label",
    }
    # TODO: Validate that the label zarr exists

    write_table(
        image_group,
        output_ROI_table,
        bbox_table,
        overwrite=overwrite,
        table_attrs=table_attrs,
    )

# New label saving
def get_zattrs(zarr_url):
    with zarr.open(zarr_url, mode="r") as zarr_img:
        return zarr_img.attrs.asdict()


def save_label_image(
    label_image, 
    label_name, 
    zarr_url, 
    label_attrs, 
    chunks = (1, 2160, 2560),
    overwrite: bool = True, 
):
    # Prepare the output label group
    # Get the label_attrs correctly (removes hack below)
    zarr_url = Path(zarr_url)
    prepare_label_group(
        image_group=zarr.group(zarr_url),
        label_name=label_name,
        overwrite=overwrite,
        label_attrs=label_attrs,
        logger=logger,
    )

    # Save label image to OME-Zarr
    label_dtype = np.uint32
    store = zarr.storage.FSStore(f"{zarr_url}/labels/{label_name}/0")
    new_label_array = zarr.create(
        shape=label_image.shape,
        chunks=chunks,
        dtype=label_dtype,
        store=store,
        overwrite=False,
        dimension_separator="/",
    )

    da.array(label_image).to_zarr(
        url=new_label_array,
    )
    logger.info(f"Saved {label_name} to OME-Zarr")
    # 4) Build pyramids for label image
    label_meta = load_NgffImageMeta(zarr_url / "labels" / label_name)
    build_pyramid(
        zarrurl=f"{zarr_url}/labels/{label_name}",
        overwrite=overwrite,
        num_levels=label_meta.num_levels,
        coarsening_xy=label_meta.coarsening_xy,
        chunksize=chunks,
        aggregation_function=np.max,
    )
    logger.info(f"Built a pyramid for the {label_name} label image")


def generate_label_zattrs_from_img_zattrs(img_attrs, label_name):
    """Hacky adaptation of zattrs."""
    # This assumes the output labels have the same shape as the loaded image
    zattrs = copy.deepcopy(img_attrs)
    label_zattrs = {}
    label_zattrs['image-label'] = {'source': {'image': '../../'}, 'version': '0.4'}
    label_zattrs['multiscales'] = [{}]
    label_zattrs['multiscales'][0]['axes'] = zattrs['multiscales'][0]['axes'][1:]
    label_zattrs['multiscales'][0]['datasets'] = zattrs['multiscales'][0]['datasets']
    # Drop channel dimension from the dataset, as labels don't have channels
    for i, dataset in enumerate(label_zattrs['multiscales'][0]['datasets']):
        dataset['coordinateTransformations'][0]['scale'] = dataset['coordinateTransformations'][0]['scale'][1:]
        label_zattrs['multiscales'][0]['datasets'][i] = dataset
    label_zattrs['multiscales'][0]['name'] = label_name
    label_zattrs['multiscales'][0]['version'] = zattrs['multiscales'][0]['version']
    return label_zattrs

### 1a) Load whole OME-Zarr image

In [ ]:
# TODO: Change to download the zenodo example data, run it on those
zarr_url = "/Users/joel/Desktop/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr/B/03/0"
level = 0
channel_index = 0

img = da.from_zarr(f"{zarr_url}/{level}")[channel_index]
zattrs = get_zattrs(zarr_url = Path(zarr_url))
img_scale = zattrs['multiscales'][0]['datasets'][level]['coordinateTransformations'][0]["scale"][1:]

### 2a) Process the image

In [ ]:
# Convert it to a numpy array, do arbitrary processing with the image
# Depending on the processing you want to do, it may also work directly in dask
min_size=256
img = np.array(img)
otsu_threshold = threshold_otsu(img)
img_thr = img > otsu_threshold
img_thr = remove_small_holes(img_thr)
img_thr_cleaned = remove_small_objects(img_thr, min_size=min_size)
label_image = label(img_thr_cleaned)
label_image.shape

### 3a) Save the resulting label image back to the OME-Zarr file

In [ ]:
new_label_name = "new_label_img_1"
label_attrs = generate_label_zattrs_from_img_zattrs(zattrs, new_label_name)
save_label_image(label_image, new_label_name, zarr_url, label_attrs)


### 1b) Load a label image

In [ ]:
zarr_url = "/Users/joel/Desktop/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr/B/03/0"
level = 0
label_name = "nuclei"

label_img = da.from_zarr(f"{zarr_url}/labels/{label_name}/{level}")
label_zattrs = get_zattrs(zarr_url = Path(zarr_url) / "labels" / label_name)
label_img_scale = label_zattrs['multiscales'][0]['datasets'][level]['coordinateTransformations'][0]["scale"]

### 2b) Edit the label image in napari

In [ ]:
# Have a look at the label image in napari
# Needs the numpy arrays, because dask arrays aren't easily edited in napari
viewer = napari.Viewer()
viewer.add_image(np.array(img), scale=img_scale)
label_layer = viewer.add_labels(np.array(label_img), scale=label_img_scale)

In [ ]:
# Optionally modify the label layer manually in napari, then get that modified label layer
label_image = label_layer.data

### 3b) Save changed label image to OME-Zarr

In [ ]:
new_label_name = "manual_label_correction_6"
save_label_image(label_image, new_label_name, zarr_url, label_zattrs)

In [ ]:
output_roi_name ="new_masking_ROI_table"

masking_roi_df = array_to_bounding_box_table(label_image, pxl_sizes_zyx=label_img_scale)
save_masking_roi_table(
    masking_roi_df=masking_roi_df, 
    zarr_url=zarr_url, 
    output_ROI_table=output_roi_name,
    output_label_name=new_label_name,
    overwrite=True
)

### 4) Save masking ROI table for the new labels

In [ ]:
new_label_name = "manual_label_correction_6"
label_attrs = get_zattrs(zarr_url = Path(zarr_url) / "labels" / label_name)
save_label_image(label_image, new_label_name, zarr_url, label_attrs)